# 오늘의 날씨를 알려주는 Agent 만들기

이 노트북은 OpenAI의 Chat Completions + Tool Calling 기능과  
OpenWeatherMap API를 이용해 **"오늘 서울 날씨 어때?"** 같은 질문에 답하는 간단한 Agent입니다.

사전 준비:
- `.env` 파일에 다음 환경 변수가 들어 있다고 가정합니다.
  - `OPENAI_API_KEY` : GPT mini용 API 키
  - `OPENWEATHER_API_KEY` : OpenWeatherMap API 키


In [6]:
import os
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI

# .env 파일에서 환경변수 로드
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

if OPENAI_API_KEY is None:
    raise ValueError("환경변수 OPENAI_API_KEY가 설정되어 있지 않습니다. .env 파일을 확인하세요.")
if OPENWEATHER_API_KEY is None:
    raise ValueError("환경변수 OPENWEATHER_API_KEY가 설정되어 있지 않습니다. .env 파일을 확인하세요.")

# OpenAI 클라이언트 생성 (gpt-mini 사용)
client = OpenAI(api_key=OPENAI_API_KEY)
MODEL = "gpt-4o-mini"

print("환경 변수 로드 및 OpenAI 클라이언트 생성 완료")


환경 변수 로드 및 OpenAI 클라이언트 생성 완료


## 1. 날씨 Tool 함수 정의

`get_today_weather(city: str)`

- 입력: 도시 이름 (예: `"Seoul"`)
- 처리: OpenWeatherMap API를 호출하여 현재 날씨 데이터를 가져옴
- 출력: 날씨 정보를 담은 **JSON 문자열**


In [7]:
def get_today_weather(city: str) -> str:
    """주어진 도시의 현재 날씨를 OpenWeatherMap에서 조회하여
    JSON 문자열 형태로 반환하는 Tool 함수입니다."""
    
    base_url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": OPENWEATHER_API_KEY,
        "units": "metric",  # 섭씨 온도
        "lang": "kr"        # 한국어 설명
    }
    
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # 에러 코드 발생 시 예외

    data = response.json()
    
    # OpenWeatherMap 응답에서 사용할 핵심 정보 추출
    weather_main = data["weather"][0]["main"]           # 날씨 상태 (Clear, Clouds 등)
    weather_description = data["weather"][0]["description"]  # 상세 설명 (맑음, 약한 비 등)
    temp = data["main"]["temp"]                         # 현재 기온 (섭씨)
    feels_like = data["main"]["feels_like"]             # 체감 온도
    humidity = data["main"]["humidity"]                 # 습도 (%)
    
    result = {
        "city": city,
        "weather": weather_main,
        "description": weather_description,
        "temp": temp,
        "feels_like": feels_like,
        "humidity": humidity,
    }
    
    # Tool의 반환값은 JSON 문자열로
    return json.dumps(result, ensure_ascii=False)

# 테스트 (직접 호출)
print(get_today_weather("Seoul"))


{"city": "Seoul", "weather": "Clouds", "description": "구름조금", "temp": 13.76, "feels_like": 12.71, "humidity": 58}


## 2. Tool JSON 스키마 정의

OpenAI Chat Completions의 `tools` 인자로 넘길 JSON 스키마입니다.


In [8]:
weather_tool_schema = {
    "type": "function",
    "function": {
        "name": "get_today_weather",
        "description": "주어진 도시의 현재 날씨 정보를 조회합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "날씨를 알고 싶은 도시 이름 (예: 'Seoul', 'Tokyo', 'New York')"
                }
            },
            "required": ["city"]
        }
    }
}

weather_tools = [weather_tool_schema]
print(json.dumps(weather_tool_schema, indent=2, ensure_ascii=False))


{
  "type": "function",
  "function": {
    "name": "get_today_weather",
    "description": "주어진 도시의 현재 날씨 정보를 조회합니다.",
    "parameters": {
      "type": "object",
      "properties": {
        "city": {
          "type": "string",
          "description": "날씨를 알고 싶은 도시 이름 (예: 'Seoul', 'Tokyo', 'New York')"
        }
      },
      "required": [
        "city"
      ]
    }
  }
}


In [9]:
# Tool 이름을 실제 파이썬 함수에 매핑
available_tools = {
    "get_today_weather": get_today_weather
}


## 3. Agent 전체 코드

1. 사용자의 질문을 메시지로 보냅니다. (예: `"오늘 서울 날씨 어때?"`)
2. GPT가 Tool을 사용할지 판단하도록 `tools`를 함께 전달합니다.
3. GPT가 Tool을 호출하겠다고 응답하면, 해당 Tool을 실제로 실행합니다.
4. Tool의 결과를 다시 GPT에 넘겨서, 자연어 최종 답변을 생성하게 합니다.


In [ ]:
# 사용자의 질문
user_question = "오늘 서울 날씨 어때?"

messages = [
    {
        "role": "system",
        "content": "너는 사용자의 질문에 대해 적절한 Tool을 사용하여 정보를 모으고, 한국어로 자연스럽게 답변하는 친절한 날씨 안내 에이전트야."
    },
    {
        "role": "user",
        "content": user_question
    }
]

print("👤 User:", user_question)

# 1차 호출: GPT가 Tool 사용 여부를 결정
response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=weather_tools,
    tool_choice="auto"
)

response_message = response.choices[0].message
messages.append(response_message)

tool_calls = response_message.tool_calls

if tool_calls:
    print("\n🤖 GPT가 Tool을 호출하기로 결정했습니다.")

    tool_messages = []
    # 각 tool_call에 대해 실제 파이썬 함수를 실행
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_tools[function_name]
        function_args = json.loads(tool_call.function.arguments)
        
        print(f"- 호출된 Tool: {function_name}({function_args})")
        
        tool_result = function_to_call(**function_args)
        
        # Tool 실행 결과를 tool 역할의 메시지로 추가
        tool_messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": tool_result,
        })
    
    # 대화 히스토리에 Tool 메시지 추가
    messages.extend(tool_messages)
    
    # 2차 호출: Tool 결과를 바탕으로 최종 자연어 답변 생성
    final_response = client.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    
    answer = final_response.choices[0].message.content
    print("\n✅ 최종 답변:")
    print(answer)
else:
    # Tool을 사용하지 않고 바로 답변한 경우
    print("\n✅ Tool 없이 바로 생성된 답변:")
    print(response_message.content)


👤 User: 오늘 서울 날씨 어때?

🤖 GPT가 Tool을 호출하기로 결정했습니다.
- 호출된 Tool: get_today_weather({'city': 'Seoul'})

✅ 최종 답변:
오늘 서울의 날씨는 구름이 조금 있는 상태입니다. 현재 기온은 약 13.8도이며, 체감 온도는 12.7도 정도 느껴집니다. 습도는 58%로 쾌적한 날씨라고 할 수 있습니다. 외출할 때는 가벼운 겉옷을 준비하시면 좋겠습니다!
